In [35]:
#Imports
from siphon.catalog import TDSCatalog
from datetime import datetime, timedelta
import xarray as xr
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import ipywidgets as widgets
import metpy.calc as mpcalc
import xesmf as xe
import xskillscore as xs
from metpy.interpolate import interpolate_to_grid, remove_nan_observations
from scipy.interpolate import griddata

In [2]:
#List the Datasets that are available to us
catalog = TDSCatalog('http://thredds.ucar.edu/thredds/catalog/catalog.xml')
catalog.catalog_refs

['Forecast Model Data', 'Forecast Products and Analyses', 'Observation Data', 'Radar Data', 'Satellite Data', 'Unidata case studies']

In [3]:
#We want the Forecast Model Data, so we will choose that link
model1_catalog = catalog.catalog_refs['Forecast Model Data'].follow()

#Create 2 ipython dropdown widgets
dropdown = widgets.Dropdown(
    #Fill with the Models to Choose From
    options=["NAM CONUS 12km from NOAAPORT","GFS Quarter Degree Forecast"],
    value="GFS Quarter Degree Forecast",
    description='Model:',
    disabled=False,
)

dropdown2 = widgets.Dropdown(
    #Leave empty for now
    options = '',
    description='Run:',
    disabled=False,
)

#Function to fill the second dropdown menu with available model times
def dropdown2_eventhandler(country):
    try:
        dropdown2.options = model1_catalog.catalog_refs[str(dropdown.value)].follow().catalog_refs
    except:
        print("Choose different file!")
    return

#Needed for the interactivity of the first dropdown for consistency
def print_pass(city):
    return

#Interactively link the widgets to functions to change the data when a new model is selected
j = widgets.interactive(print_pass,city=dropdown2)
i = widgets.interactive(dropdown2_eventhandler, country=dropdown)

#Display the widgets
display(i)
display(j)

interactive(children=(Dropdown(description='Model:', index=1, options=('NAM CONUS 12km from NOAAPORT', 'GFS Qu…

interactive(children=(Dropdown(description='Run:', options=['GFS_Global_0p25deg_20210118_0000.grib2', 'GFS_Glo…

In [5]:
#Grab the catalog of data from the model we chose
m1_catalog = model1_catalog.catalog_refs[str(dropdown.value)].follow()

#Grab the Dataset for the model run time we selected
m1 = m1_catalog.catalog_refs[str(dropdown2.value)].follow()

#Open the Dataset in xarray and parse it with cf conventions
#This will allow us to add in any missing coordinates with metpy
ds1 = xr.open_dataset(str(m1.datasets[0].access_urls['OPENDAP'])).metpy.parse_cf()

#Now we try and find the times, these two models have either time1 or time2 depending on the variable
try:
    times = ds1.time2[ds1.time2<np.datetime64('now')]
except:
    times = ds1.time1[ds1.time1<np.datetime64('now')]

#Get a list of variables for the models
variables_temp = list(ds1.data_vars)
variables = []

#For now, this only concerns 2-meter temperatures, so only have those as options
[variables.append(x) for x in variables_temp if "Temperature_height" in x]

#Create two ipython dropdown widgets for the model forecast times we can choose from and the variables
time_dropdown = widgets.Dropdown(
    options= list(times[:].values),
    description='Time:',
    disabled=False,
)
variable_dropdown = widgets.Dropdown(
    options= variables,
    description='Variable:',
    disabled=False,
)

#Display the widgets
display(time_dropdown)
display(variable_dropdown)

Dropdown(description='Time:', options=(numpy.datetime64('2021-01-18T00:00:00.000000000'), numpy.datetime64('20…

Dropdown(description='Variable:', options=('Temperature_height_above_ground',), value='Temperature_height_abov…

In [33]:
temp.longitude

<xarray.DataArray 'longitude' (y: 1377, x: 2145)>
array([[-121.55400107, -121.53021304, -121.50642335, ...,  -69.25584397,
         -69.23199966,  -69.20816166],
       [-121.55872141, -121.53492937, -121.51113567, ...,  -69.25126048,
         -69.22741213,  -69.20357009],
       [-121.56344338, -121.53964733, -121.51584962, ...,  -69.2466754 ,
         -69.22282301,  -69.19897694],
       ...,
       [-130.08710771, -130.05627705, -130.02544269, ...,  -60.96337083,
         -60.93241604,  -60.90147098],
       [-130.09527046, -130.06443329, -130.0335924 , ...,  -60.9554312 ,
         -60.92446981,  -60.89351815],
       [-130.10343773, -130.07259403, -130.04174662, ...,  -60.94748716,
         -60.91651917,  -60.88556091]])
Coordinates:
    reftime    datetime64[ns] 2021-01-18
  * x          (x) float32 -2.763e+06 -2.761e+06 ... 2.679e+06 2.682e+06
  * y          (y) float32 -2.638e+05 -2.612e+05 ... 3.228e+06 3.231e+06
    metpy_crs  object Projection: lambert_conformal_conic
    latitude   (y, x) float64 20.19 20.2 20.2 20.21 ... 50.12 50.12 50.11 50.11
    longitude  (y, x) float64 -121.6 -121.5 -121.5 ... -60.95 -60.92 -60.89
Attributes:
    units:          degrees_east
    standard_name:  longitude

In [37]:
#Grab the observation data from the THREDDS catalog
obs_catalog = catalog.catalog_refs['Forecast Products and Analyses'].follow()
obs = obs_catalog.catalog_refs["Real Time Mesoscale Analysis 2.5 km"].follow()

#Filter the observations to get the time closest to the model forecast that was chosen
obs = obs.catalog_refs.filter_time_nearest(datetime.utcfromtimestamp(time_dropdown.value.astype('O')/1e9)).follow()

#Open the dataset in xarray with cf standards 
obs_data = xr.open_dataset(
    str(obs.datasets[0].access_urls['OPENDAP'])
).metpy.parse_cf()

#This does not have lat/lon coords, so add them in
obs_data = obs_data.metpy.assign_latitude_longitude()

#Pull out the 2-meter temperature data
temp = obs_data["Temperature_Analysis_height_above_ground"]

#Interpolate from lambert conformal to lat_lon grid
lon, lat = temp.longitude.values, temp.latitude.values
xp, yp, _ = temp[0,0].metpy.cartopy_crs.transform_points(ccrs.PlateCarree(), lon, lat).T
griddata((temp.x.values,temp.y.values), temp[0,0].values, (xp, yp), method='nearest')
#Now check if our model forecast dataset contains lat/lon coords, if not, add them in
# if "lat" in list(ds1[variable_dropdown.value].coords):
#     ds_out = xr.Dataset({'lat': ds1.lat,
#                      'lon': ds1.lon,
#                     }
#                    )
# else:
#     ds1 = ds1.metpy.assign_latitude_longitude()
#     ds1 = ds1.rename({"longitude": "lon","latitude": "lat"})
#     ds_out = xr.Dataset({'lat': ds1.lat,
#                      'lon': ds1.lon,
#                     }
#                    )
    
# #The observations are on a different size grid compared to the model forecasts!
# #We will used XESMF-python package to regrid the observations to the model grid

# #To regrid, we need to create a Regridder function using the grid from the model
# #Regridder(our data, grid dimensions to regrid to, type of regridding algorithm)
# #We have a try statement as the GFS and NAM grids are setup differently
# try:
#     regridder = xe.Regridder(temp, ds_out.sel(lat=slice(55,20),lon=slice(230,300)), 'bilinear')
# except:
#     regridder = xe.Regridder(temp, ds_out, 'bilinear')  
    
# #Once the function is created, we can regrid our data
# dr_out = regridder(temp[0,0])


ValueError: shape mismatch: objects cannot be broadcast to a single shape

In [ ]:
#Now we need to compare the two datasets, so we will subtract them from each other
#Since they can have different time coordinates, we need to check for each one separately
if "time" in list(ds1[variable_dropdown.value].coords):
    im_data = ds1[variable_dropdown.value].sel(time=time_dropdown.value)[0] - dr_out
elif "time1" in list(ds1[variable_dropdown.value].coords):
    im_data = ds1[variable_dropdown.value].sel(time1=time_dropdown.value)[0] - dr_out
elif "time2" in list(ds1[variable_dropdown.value].coords):     
    im_data = ds1[variable_dropdown.value].sel(time2=time_dropdown.value)[0] - dr_out
else:
    print("Unable to Find Time Coordinate!")

#Once we have the anomaly dataset, we can plot it
#Since the two grids are different, we will have two plotting functions separated by the try statement
try: #For GFS
    ax = plt.axes(projection=ccrs.LambertConformal())    
    im_data.sel(lat=slice(50,23), lon=slice(238,290)).plot.pcolormesh(transform=ccrs.PlateCarree(),vmin = -10, vmax = 10, extend = 'both', cbar_kwargs={
        "orientation": "horizontal",
        "label": "2-meter Temperature Anomaly (C)",
        
    })  # passed to plt.colorbar)
    ax.add_feature(cfeature.STATES.with_scale('10m'))
    plt.title("GFS (0.25 degree) Forecast - Observation")
    
except:#For NAM
    ax = plt.axes(projection = ccrs.LambertConformal())
    ax.set_extent([-120,-72,24,50],ccrs.PlateCarree())
    im_data.plot.pcolormesh("lon","lat",transform=ccrs.PlateCarree(),vmin = -10, vmax = 10, extend = 'both', cbar_kwargs={
        "orientation": "horizontal",
        "label": "2-meter Temperature Anomaly (C)",
        
    })  # passed to plt.colorbar)
    ax.add_feature(cfeature.STATES.with_scale('10m'))
    plt.title("NAM 12km Forecast - Observation")
    


In [ ]:
#Now set up the skill score parameters
#First we put our two grids into new variables
#Note that for xskillscore to work on each grid point, we need an arbitrary first dimension to work over
#Here we assign 'time' to that dimension with a length of 1 since both grids are at the same time
if "time" in list(ds1[variable_dropdown.value].coords):
    grid_model = ds1[variable_dropdown.value].sel(time=time_dropdown.value)[0].sel(lat=slice(55,20),lon=slice(230,300)).assign_coords(time=dr_out.time, force=true).expand_dims('time')
elif "time1" in list(ds1[variable_dropdown.value].coords):
    grid_model = ds1[variable_dropdown.value].sel(time1=time_dropdown.value)[0].sel(lat=slice(55,20),lon=slice(230,300)).assign_coords(time=dr_out.time).expand_dims('time')
elif "time2" in list(ds1[variable_dropdown.value].coords):     
    grid_model = ds1[variable_dropdown.value].sel(time2=time_dropdown.value)[0].sel(lat=slice(55,20),lon=slice(230,300)).assign_coords(time=dr_out.time).expand_dims('time')
else:
    print("Unable to Find Time Coordinate!")
grid_obs = dr_out.expand_dims('time')

In [ ]:
temp[0,0].values.shape

In [ ]:

# x_masked, y_masked, temps = remove_nan_observations(xp.T, yp.T, temp[0,0].values)

# tgridx, tgridy, stemp = interpolate_to_grid(x_masked, y_masked, temps, interp_type='cressman',
#                                               minimum_neighbors=1, search_radius=400000,
#                                               hres=100000)

In [ ]:
temps.shape